In [ ]:
# multiple clusters
# each cluster has edge devices with x,y coordinates
# conditional probability for a cluster = (cluster probability) / (sum of all cluster probabilities)
# cluster probability = e^(-gth * di)
# di -> cluster and mobile device (distance)
# sum -> 1000 terms
# +1 -> common difference, ni(t), ni(t) + 1, ni(t) + 2
# ni(t) = 10
# delay tolerance -> 1sec
# Gth -> 10
# denominator = sum of numerator (i!=j)
# rd -> distance threshold 
# set of clusters dist <= rd
# cluster centroid 
# time global var 

#   Task5       Task1 , Task2 -> C1
#   Task6       Task3 -> C2 -> Task5
#          Task4 -> C3

# (k , k + 1/p) -> busy cluster
# t = 0, t = k, t = 2 * k

In [2]:
pip install import-ipynb



[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
import import_ipynb


In [13]:
import constants
import edge_device_selection

importing Jupyter notebook from edge_device_selection.ipynb


In [12]:
import math

In [11]:
factorial = [1]
def calculate_factorial():
    for i in range(1,20 + 1):
        factorial.append(factorial[i - 1] * i)

In [10]:
# [[], []] -> edge cluster x coordinates, [[], []] -> mobile device
# e^(-nc * t) * ((nc * t)^nc / nc!)
def probability_cluster_task(edge_device_cluster_coordinates, mobile_device_coordinates):
    power_gain_threshold = 10
    centroid_x_coordinate = sum(edge_device_cluster_coordinates[0]) / len(edge_device_cluster_coordinates[0])
    centroid_y_coordinate = sum(edge_device_cluster_coordinates[1]) / len(edge_device_cluster_coordinates[1])
    distance_cluster_mobile_device = math.sqrt((centroid_x_coordinate-mobile_device_coordinates[0]) * (centroid_x_coordinate-mobile_device_coordinates[0]) + (centroid_y_coordinate-mobile_device_coordinates[1]) * (centroid_y_coordinate-mobile_device_coordinates[1]))
    probability_part_1 = pow(math.e, (power_gain_threshold * distance_cluster_mobile_device))
    probability_part_2 = 0
    number_of_computing_tasks_at_cluster = 10
    delay_tolerance = 1
    for i in range(0, 20 + 1):
        probability_part_2 += pow(math.e, -(number_of_computing_tasks_at_cluster + i) * delay_tolerance) * ((pow((number_of_computing_tasks_at_cluster + i) * delay_tolerance, (number_of_computing_tasks_at_cluster + i))) / factorial[number_of_computing_tasks_at_cluster + i])
    probability_final = probability_part_1 * probability_part_2
    return probability_final

In [9]:
def cluster_assigning(threshold,offloading_factor_value):
    # 10 * 10 matrix
    mobile_device_cluster_probability = []
    for i in range(0,10):
        for j in range(0,10):
            probability_mobile_device_each_cluster = []
            for k in range(0,10):
                probability_mobile_device_each_cluster.append(probability_cluster_task([edge_device_cluster_x_coordinates[j][k],edge_device_cluster_y_coordinates[j][k]], [mobile_device_x_coordinates[i],mobile_device_y_coordinates[i]]))
            mobile_device_cluster_probability.append(probability_mobile_device_each_cluster)


    task_interval = 10
    f = 1
    for time_interval in range(0,10):
        current_time = task_interval * time_interval
        assigned_tasks_to_cluster = []

        assigned = False

        for i in range(0,10):
            assigned_tasks_to_cluster.append([])

        for mobile_device in range(0,10 * f):
            max_probability = 0
            selected_cluster = -1
            for edge_device_cluster in range(0,10):
                if current_time < free_cluster_next_time[edge_device_cluster]:
                    continue
                if (max_probability < mobile_device_cluster_probability[mobile_device % 10][edge_device_cluster]):
                    max_probability = mobile_device_cluster_probability[mobile_device % 10][edge_device_cluster]
                    selected_cluster = edge_device_cluster
            if selected_cluster != -1:
                assigned = True
                assigned_tasks_to_cluster[selected_cluster].append(mobile_device)
            else:
                f = f + 1
                break

            if(assigned):
                f = 1


        for i in range(0,len(assigned_tasks_to_cluster)):
            if (len(assigned_tasks_to_cluster[i]) > 0):
                mobile_device_local_coordinate_x = [], mobile_device_local_coordinate_y = []
                for j in range(0,len(assigned_tasks_to_cluster[i])):
                    mobile_device_local_coordinate_x.append(mobile_device_x_coordinates[assigned_tasks_to_cluster[i][j]])
                    mobile_device_local_coordinate_y.append(mobile_device_y_coordinates[assigned_tasks_to_cluster[i][j]])
                local_time = edge_device_selection(edge_device_cluster_x_coordinates[i],edge_device_cluster_y_coordinates[i],mobile_device_local_coordinate_x,mobile_device_local_coordinate_y,threshold,offloading_factor_value,mobile_device_computing_power,mobile_device_transmission_power,task_bits,CPU_cycle_task_bit,execution_deadline,edge_device_computing_power)
                free_cluster_next_time[i] = current_time + task_interval + local_time + 1

            
